In [1]:
import jax
jax.config.update("jax_platform_name", "cpu")
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp
import numpy as np
import spax

# Fisher information
Let $p(\mathbf{d} \in D | \boldsymbol{\theta})$ be a pdf parameterized with $\boldsymbol{\theta}$. Fisher matrix is then defined as:
$$\mathcal{F}_{ij} = \operatorname{E}\left[\left.\left(\frac{\partial}{\partial\theta_i} \log p(D|\boldsymbol\theta)\right)\left(\frac{\partial}{\partial\theta_j} \log p(D|\boldsymbol\theta)\right)\right|\boldsymbol\theta\right] = -\operatorname{E}\left[\left.\frac{\partial^2}{\partial\theta_i\, \partial\theta_j} \log p(D|\boldsymbol\theta)\right|\boldsymbol\theta\right]\,.$$

## Gaussian-distributed data
In the simplest case, we can assume multivariate gaussian as the underlying pdf, $\log p(\mathbf{d}|\boldsymbol\theta) = - 1/2\,  (\mathbf{d} - \boldsymbol\mu(\boldsymbol\theta))^\textsf{T} \, \Sigma^{-1} \, (\mathbf{d} - \boldsymbol\mu(\boldsymbol\theta))$, where we assumed that covariance matrix doesn't depend on $\boldsymbol\theta$.

Now from equation above it follows: $$ \mathcal{F}_{ij} = \frac{\partial\boldsymbol\mu^\textsf{T}}{\partial\theta_i}\Sigma^{-1}\frac{\partial\boldsymbol\mu}{\partial\theta_j}$$
If $X$ is $n \times m$ matrix, where $n$ represents dimensionality of the data and $m$ number of samples, then covariance matrix is simply $$\Sigma = \frac{1}{m-1} X \, X^T \, ,$$
with the assumption that the mean was removed, i.e. `X -= X.mean(axis = 1)`.

On the other hand, if $X_i^{+}$ and $X_i^{-}$ are $n \times m'$ matrices representing a set of data points with distance $\Delta\theta_i$ apart, then:
$$\frac{\partial\boldsymbol\mu}{\partial\theta_i} \approx \frac{1}{m'}\sum_k \frac{\mathbf{d}_{ki}^{+} - \mathbf{d}_{ki}^{-}}{\Delta\theta_i} \, ,$$
where $\mathbf{d}_{ki}^{+, -}$ are columns of $X_i^{+, -}$.

## Need for compression
In the case $n \ge m$, covariance matrix is non invertible and some data compression is needed. Here we are implementing a simple PCA compression of order $N$: $ \widetilde{X}_N = U_N^T \, X$, where $\sim$ denotes compressed space and $U_N$ is $n \times N$ rotation matrix.

Covariance matrix can now be written as
$$ \widetilde{\Sigma}_N = \frac{1}{m-1} \widetilde{X}_N \, \widetilde{X}_N^T = \frac{1}{m-1} U_N^T \, X \, X^T U_N = \sigma_N^2 \, ,$$
where $\sigma_N^2$ is diagonal matrix containing first $N$ principal components of the covariance matrix.

If we denote $J \equiv \partial\boldsymbol\mu / \partial\boldsymbol\theta$, one can easily show $\widetilde{J}_N = U_N^T J$. Therefore, if we start with $F = J^T \Sigma^{-1} J$, after PCA compression one has:
$$ F_N = J^T \, U_N  \, \sigma_N^{-2} \, U_N^T J  \, .$$

### Extras
$$ X_N = U_N \, U_N^T \, X$$
$$ \Sigma_N \equiv \frac{1}{m-1} \, X_N \, X_N^T = \frac{1}{m-1} \, U_N \, U_N^T \, X \, X^T \, U_N \, U_N^T = U_N \, \sigma_N^2 \, U_N^T $$
$$ \Sigma_N^{-1} \equiv U_N \, \sigma_N^{-2} \, U_N^T $$
With this in mind and the fact that $U_N^T \, U_N = I$, we can see that 
$$ \Sigma_N^{-1} \Sigma_N \Sigma_N^{-1} = \Sigma_N^{-1} \, ,$$ 
$$ \Sigma_N \Sigma_N^{-1} \Sigma_N = \Sigma_N  \, ,$$ 
i.e. it represents Moore-Penrose inverse. Moreover,
$$ F_N = J^T \, \Sigma_N^{-1} \, J \, .$$

## Testing the code

### Gaussian example
If we take likelihood of the data point $\mathbf{d}$ of dimensionality $n$ as:
$$ \mathcal{L}(\mathbf{d} | \sigma) = \frac{1}{(\sigma\sqrt{2\pi})^n} \exp\left({-\frac{\mathbf{d} \cdot \mathbf{d}}{2\sigma^2}}\right) \, ,$$
$$ \ln \mathcal{L}(\mathbf{d} | \sigma)  = -n\ln(\sigma) - \frac{1}{2}\frac{\mathbf{d} \cdot \mathbf{d}}{\sigma^2} -\frac{n}{2}\ln 2\pi\, .$$

One can show that such likelihood has maximum for $s \equiv \mathbf{d} \cdot \mathbf{d} = n\sigma^2$, where $s$ represents summary. Calculating Fisher information around that summary, it amounts to $F = 2 n/ \sigma_{\text{fid}}^2$. If we take fiducial value $\sigma_{\text{fid}} = 1$, $F = 2n$.

In [2]:
@jax.jit
def summary(x):
    return jnp.sum(x * x, keepdims = True)

N_samples, N_dim = (10000, 4)
random_sample = jax.random.normal(jax.random.PRNGKey(1), shape = (N_dim, N_samples))
sigma_fid = 1.
δθ = 1e-6 * sigma_fid
summary_data = jax.vmap(summary, in_axes = 1, out_axes = 1)(random_sample * sigma_fid)
summary_data_plus = jax.vmap(summary, in_axes = 1, out_axes = 1)(random_sample * (sigma_fid + δθ / 2))
summary_data_minus = jax.vmap(summary, in_axes = 1, out_axes = 1)(random_sample * (sigma_fid - δθ / 2))
summary_derivative = jnp.stack([summary_data_minus, summary_data_plus], axis=0)[jnp.newaxis, ...]
print(summary_data.shape, summary_derivative.shape)

Fisher = spax.Fisher()
Fisher.fit(summary_data, summary_derivative, δθ, batch_size = 1)
# F = []
# for n in range(1, N_dim):
#     F.append(float(Fisher.compute(N = n)))
# print(F)
float(Fisher.compute()), 2 * N_dim/ sigma_fid**2

(1, 10000) (1, 2, 1, 10000)


(8.031949043273926, 8.0)

In [3]:
# same example as before, but keeping N_dim summaries

@jax.jit
def summary(x):
    return x * x

N_samples, N_dim = (10000, 4)
random_sample = jax.random.normal(jax.random.PRNGKey(1), shape = (N_dim, N_samples))
sigma_fid = 1.
δθ = 1e-6 * sigma_fid
summary_data = jax.vmap(summary, in_axes = 1, out_axes = 1)(random_sample * sigma_fid)
summary_data_plus = jax.vmap(summary, in_axes = 1, out_axes = 1)(random_sample * (sigma_fid + δθ / 2))
summary_data_minus = jax.vmap(summary, in_axes = 1, out_axes = 1)(random_sample * (sigma_fid - δθ / 2))
summary_derivative = jnp.stack([summary_data_minus, summary_data_plus], axis=0)[jnp.newaxis, ...]
summary_derivative = jnp.swapaxes(summary_derivative, 0, 2)
print(summary_data.shape, summary_derivative.shape)

Fisher = spax.Fisher()
Fisher.fit(summary_data, summary_derivative, δθ, batch_size = 1)
# F = []
# for n in range(1, N_dim):
#     F.append(float(Fisher.compute(N = n)))
# print(F)
float(Fisher.compute()), 2 * N_dim / sigma_fid**2

(4, 10000) (4, 2, 1, 10000)


(8.033331871032715, 8.0)

In a more complicated case, where we allow $\sigma$ to vary from dimension to dimension, and we allow summary to be $s_i = d_i^2$, we have that Fisher matrix will be $F_{ij} = \delta_{ij} \cdot 2/\sigma^2_{i, \text{fid}}$ and Fisher information is equal to $F = \Pi_{i} 2 / \sigma^2_{i, \text{fid}}$. In the following example we fix fiducial variances so that $1 / \sigma^2_{i, \text{fid}} = i$, which amounts to $F = 2^n \cdot n!$.

In [4]:
@jax.jit
def summary(x):
    return x * x

N_samples, N_dim = (1000000, 4)
random_sample = jax.random.normal(jax.random.PRNGKey(1), shape = (N_dim, N_samples))
# making sigma so that fisher will be twice the sum of numbers up to N_dim
sigma_fid = jnp.sqrt(1 / jnp.arange(1, N_dim + 1))[:, jnp.newaxis]
# sigma_fid = jnp.sqrt(1 / jnp.ones(N_dim))[:, jnp.newaxis]
δθ = 1e-6 * sigma_fid
δθ_const = 1e-6
summary_data = summary(random_sample * sigma_fid)
summary_derivative = []
for i, δθi in enumerate(δθ):
    delta = np.zeros(shape = (len(δθ), 1))
    # delta[i, 0] = δθ_const
    delta[i, 0] = δθi
    plus = summary(random_sample * (sigma_fid + delta / 2))
    minus = summary(random_sample * (sigma_fid - delta / 2))
    summary_derivative.append(
        jnp.stack([minus, plus], axis=0)
    )
summary_derivative = jnp.stack(summary_derivative, axis = 0)
summary_derivative = jnp.swapaxes(summary_derivative, 0, 2)
print(summary_data.shape, summary_derivative.shape)

Fisher = spax.Fisher()
Fisher.fit(summary_data, summary_derivative, δθ, batch_size = N_dim)
detF, F = Fisher.compute(return_matrix = True)
print(F) 
float(detF), 2**N_dim * np.math.factorial(N_dim)

(4, 1000000) (4, 2, 4, 1000000)
[[ 2.0054202e+00 -2.0387049e-03 -1.1878407e-03 -6.0898829e-03]
 [-2.0387049e-03  4.0167580e+00  5.2774521e-03 -1.0410335e-02]
 [-1.1878407e-03  5.2774521e-03  5.9966693e+00 -1.0191760e-02]
 [-6.0898829e-03 -1.0410335e-02 -1.0191760e-02  8.0004845e+00]]


(386.45892333984375, 384)